# Zadanie - Agent pogodowy z DSPy ReAct - Rozwiązanie

Twoim zadaniem jest stworzenie agenta pogodowego używając DSPy z modułem `dspy.ReAct()`. Agent ma być asystentem odpowiadającym na pytania o pogodę w danym mieście.

## Wymagania

1. **Wykorzystaj funkcje z przykładu function calling** (003-01. LLM-function_calling.ipynb):
   - `get_geolocation(location)` - pobiera współrzędne geograficzne dla danej nazwy
   - `get_current_weather(latitude, longitude)` - pobiera aktualne dane pogodowe

2. **Skonfiguruj DSPy** zgodnie z przykładem z 009-01. DSPy.ipynb:
   - Użyj modelu GPT-4.1

3. **Stwórz agenta ReAct** który:
   - Przyjmuje pytanie o pogodę jako input
   - Zwraca odpowiedź tekstową z informacją o pogodzie
   - Używa dostępnych narzędzi do zebrania danych

4. **Przetestuj agenta** na przykładowych pytaniach

5. Opcjonalnie, **podejrzyj prompty wysłane przez DSPy** używając `dspy.inspect_history(n=4)`


## Wskazówki

- Dokumentacja DSPy.ReAct(): https://dspy.ai/tutorials/agents
- Wykorzystaj sygnaturę DSPy do zdefiniowania wejścia i wyjścia agenta

## Importy

In [ ]:
# Import bibliotek
import dspy
import os
import requests

## Funkcje narzędzi

Te same co w `003-01. LLM-function_calling.ipynb`

In [ ]:
def get_geolocation(location):
    """
    Pobiera współrzędne geograficzne oraz dane lokalizacyjne.

    Args:
        location (str): Nazwa lokalizacji, dla której chcemy uzyskać współrzędne geograficzne.

    Returns:
        dict: Dane lokalizacyjne w formacie JSON.
    """
    # Definiowanie endpointu i parametrów dla API OpenStreetMap Nominatim
    geocode_endpoint = "https://nominatim.openstreetmap.org/search"
    geocode_params = {
        "q": location,
        "format": "json"
    }

    # Nagłówek User-Agent jest wymagany do korzystania z tego API
    headers = {
        "User-Agent": "Python script"
    }

    # Wykonanie zapytania
    geocode_response = requests.get(geocode_endpoint, params=geocode_params, headers=headers)

    # Wyciągnięcie współrzędnych geograficznych
    geocode_data = geocode_response.json()
    
    # Zwrócenie tylko nazwy miejsca, szerokości i długości geograficznej
    simplified_data = {
        "name": geocode_data[0]["display_name"],
        "latitude": geocode_data[0]["lat"],
        "longitude": geocode_data[0]["lon"]
    }
    return simplified_data

In [ ]:
def get_wind_direction(degrees):
    """
    Konwertuje kierunek wiatru z stopni na nazwy kierunków.

    Args:
        degrees (float): Kierunek wiatru w stopniach.

    Returns:
        str: Nazwa kierunku wiatru.
    """
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
                  'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    index = int((degrees + 11.25) // 22.5) % 16
    return directions[index]

def get_current_weather(latitude, longitude):
    """
    Pobiera aktualne dane pogodowe dla podanych współrzędnych geograficznych.

    Args:
        latitude (float): Szerokość geograficzna.
        longitude (float): Długość geograficzna.

    Returns:
        dict: Dane pogodowe w formacie JSON.
    """
    # Zdefiniowanie endpointu i parametrów dla Open Meteo API
    weather_endpoint = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True
    }

    # Wykonanie requestu
    weather_response = requests.get(weather_endpoint, params=weather_params)

    # Wyciągnięcie informacji o pogodzie
    weather_data = weather_response.json()

    simplified_weather = {
        "temperature": f"{weather_data['current_weather']['temperature']} °C",
        "wind_speed": f"{weather_data['current_weather']['windspeed']} km/h",
        "wind_direction": get_wind_direction(weather_data['current_weather']['winddirection']),
        "is_day": "day" if weather_data['current_weather']['is_day'] else "night"
    }

    return simplified_weather

## Rozwiązanie zadania